<a href="https://colab.research.google.com/github/unique-subedi/gene-expression/blob/main/Vinod_gene_expression_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import os
import datetime
import time
import math
import numpy as np
from numpy import linalg as LA
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.stats.diagnostic import het_white
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
!pip install qnorm
import qnorm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pyreadr
import pyreadr
urllib.request.urlretrieve("https://raw.githubusercontent.com/unique-subedi/gene-expression/main/data/brain.rda", "brain.rda")
brain = pyreadr.read_r("brain.rda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 363 kB 29.0 MB/s 


In [4]:
expression = pd.DataFrame(brain["expression"])
genes = pd.DataFrame(brain["genes"])
samples = pd.DataFrame(brain["samples"])

In [5]:
ACC = "A.C. cortex"
CER = "cerebellum"
DLPFC = "D.L.P.F. cortex"


# Compute Bacterial Genes

In [90]:
# Need to compute all bacteria genes. Contains AFFX, does Not have HUM
genes.at["AFFX-BioDn-3_at", 'chrom'] = float("Nan")
genes_control = genes[['AFFX' in s for s in genes.index]]
genes_control = genes_control[['HUM' not in s for s in genes_control.index]]
genes_control = genes_control[['hum' not in s for s in genes_control.index]]
genes_bac_index = genes_control[genes_control.chrom.isnull() & genes_control.sym.isnull()].index
print(genes_bac_index)


Index(['AFFX-BioB-3_at', 'AFFX-BioB-3_st', 'AFFX-BioB-5_at', 'AFFX-BioB-5_st',
       'AFFX-BioB-M_at', 'AFFX-BioB-M_st', 'AFFX-BioC-3_at', 'AFFX-BioC-3_st',
       'AFFX-BioC-5_at', 'AFFX-BioC-5_st', 'AFFX-BioDn-3_at',
       'AFFX-BioDn-3_st', 'AFFX-BioDn-5_at', 'AFFX-BioDn-5_st',
       'AFFX-CreX-3_at', 'AFFX-CreX-3_st', 'AFFX-CreX-5_at', 'AFFX-CreX-5_st',
       'AFFX-DapX-3_at', 'AFFX-DapX-5_at', 'AFFX-DapX-M_at', 'AFFX-LysX-3_at',
       'AFFX-LysX-5_at', 'AFFX-LysX-M_at', 'AFFX-M27830_3_at',
       'AFFX-M27830_5_at', 'AFFX-M27830_M_at', 'AFFX-MurFAS_at',
       'AFFX-MurIL10_at', 'AFFX-MurIL2_at', 'AFFX-MurIL4_at', 'AFFX-PheX-3_at',
       'AFFX-PheX-5_at', 'AFFX-PheX-M_at', 'AFFX-ThrX-3_at', 'AFFX-ThrX-5_at',
       'AFFX-ThrX-M_at', 'AFFX-TrpnX-3_at', 'AFFX-TrpnX-5_at',
       'AFFX-TrpnX-M_at', 'AFFX-YEL002c/WBP1_at', 'AFFX-YEL018w/_at',
       'AFFX-YEL021w/URA3_at', 'AFFX-YEL024w/RIP1_at'],
      dtype='object', name='rownames')


# Genes to Probe Set

In [89]:
genes_sym = genes[genes.sym.notnull()]
unique_genes = genes_sym["sym"].unique()
gene_to_probe_set = {}

for gene in unique_genes:
  gene_to_probe_set[gene] = genes_sym.index[genes_sym.sym == gene].tolist()

# Normalize Data

In [88]:
genes_Y_crom = genes[genes.chrom == "Y"].index
mean_bac_exp = expression[genes_bac_index].mean(axis=1)
std_bac_exp = expression[genes_bac_index].std(axis=1)
expression_norm =  (expression - mean_bac_exp.values[:, None]).divide(std_bac_exp, axis=0)
expression_qnorm = qnorm.quantile_normalize(expression, axis=0)
expression_ecdf = expression.copy()
for i in range(len(expression)):
  ecdf = ECDF(expression_ecdf.iloc[i, :].values)
  expression_ecdf.iloc[i, :] = ecdf(expression_ecdf.iloc[i, :].values)

# Hypothesis Testing

In [64]:
def compute_top_genes_tissue(data, tissue):
  express_norm_samp = pd.concat([data, samples], axis=1)
  express_samp_loc = express_norm_samp

  df_male = express_samp_loc[(express_samp_loc.region == tissue) & (express_samp_loc.sex == "male")]
  df_female = express_samp_loc[(express_samp_loc.region == tissue) & (express_samp_loc.sex == "female")]

  unique_probe_set = data.columns
  probe_set_to_pvalue = {}

  for probe_set in unique_probe_set:
    stat, p = stats.ks_2samp(data.loc[df_male.index][probe_set], data.loc[df_female.index][probe_set])
    probe_set_to_pvalue[probe_set] = p

  gene_p_values = []

  for gene in genes_sym["sym"].unique():
    gene_p = 0
    gene_probes = gene_to_probe_set[gene]
    for probe in gene_probes:
      gene_p = max(gene_p, probe_set_to_pvalue[probe])
    gene_p_values.append((gene_p, gene))
    
  sorted_pval = sorted(gene_p_values, key=lambda tup: tup[0])
  topgenes = [tup[1] for tup in sorted_pval[:10]]
  topchrom = [genes[genes.sym == tg]["chrom"][0] for tg in topgenes]
  chrom_counter = Counter(topchrom)
  count = np.sum([chrom_counter[key] for key, value in chrom_counter.items() if key in ["X", "Y", "X Y"]])
  return topgenes, topchrom, count

In [65]:
topgenes, topchrom, cnt = compute_top_genes_tissue(expression_norm, ACC)
print(topgenes)
print(topchrom)
print(cnt)

['DDX3Y', 'RPS4Y1', 'UTY', 'XIST', 'USP9Y', 'KDM5D', 'EIF1AY', 'SNRNP35', 'MOBP', 'SCGB1D1']
['Y', 'Y', 'Y', 'X', 'Y', 'Y', 'Y', '12', '3', '11']
7


In [66]:
topgenes, topchrom, cnt = compute_top_genes_tissue(expression_norm, CER)
print(topgenes)
print(topchrom)
print(cnt)

['DDX3Y', 'RPS4Y1', 'KDM5D', 'XIST', 'UTY', 'CD24', 'TTTY15', 'USP9Y', 'DUSP5', 'GEMIN4']
['Y', 'Y', 'Y', 'X', 'Y', '6', 'Y', 'Y', '10', '17']
7


In [67]:
topgenes, topchrom, cnt = compute_top_genes_tissue(expression_norm, DLPFC)
print(topgenes)
print(topchrom)
print(cnt)

['DDX3Y', 'RPS4Y1', 'USP9Y', 'XIST', 'UTY', 'KDM5D', 'HNRNPF', 'TTTY15', 'NLGN4Y', 'HIST1H3I']
['Y', 'Y', 'Y', 'X', 'Y', 'Y', '10', 'Y', 'Y', '6']
8


#OLS

In [98]:
human_probes = genes[genes.chrom.notnull()].index;
print(human_probes)
coef_gene = []
num_sig = 0
alpha = 0.05
for probe in human_probes[:]:
  samples_tmp = samples.drop(columns= ["patient"]);
  one_hot_encoding = pd.get_dummies(samples_tmp);
  one_hot_encoding = one_hot_encoding.drop(columns= ["sex_female", 'chip.version_v2', "region_cerebellum", "lab_Davis"]);
  # one_hot_encoding = one_hot_encoding.drop(columns= ["sex_female", 'chip.version_v2', "patient_patient_10", "region_cerebellum", "lab_Davis"]);
  X = sm.add_constant(one_hot_encoding);
  model = sm.OLS(expression_norm[probe],X).fit();
  white_test = het_white(model.resid,  model.model.exog)

  #define labels to use for output of White's test
  labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']

  #print results of White's test
  p = dict(zip(labels, white_test))["Test Statistic p-value"]
  if p <= alpha:
    num_sig += 1
print(num_sig)




Index(['1000_at', '1001_at', '1002_f_at', '1003_s_at', '1004_at', '1005_at',
       '1006_at', '1007_s_at', '1008_f_at', '1009_at',
       ...
       'AFFX-HUMGAPDH/M33197_5_st', 'AFFX-HUMGAPDH/M33197_M_at',
       'AFFX-HUMGAPDH/M33197_M_st', 'AFFX-HUMISGF3A/M97935_3_at',
       'AFFX-HUMISGF3A/M97935_5_at', 'AFFX-HUMISGF3A/M97935_MA_at',
       'AFFX-HUMISGF3A/M97935_MB_at', 'AFFX-HUMTFRR/M11507_3_at',
       'AFFX-HUMTFRR/M11507_5_at', 'AFFX-HUMTFRR/M11507_M_at'],
      dtype='object', name='rownames', length=11728)
5891


In [ ]:
human_probes = genes[genes.chrom.notnull()].index;
print(human_probes)
coef_gene = []
num_sig = 0
alpha = 0.05
for probe in human_probes[:1]:
  samples_tmp = samples.drop(columns= ["patient"]);
  one_hot_encoding = pd.get_dummies(samples_tmp);
  one_hot_encoding = one_hot_encoding.drop(columns= ["sex_female", 'chip.version_v2', "region_cerebellum", "lab_Davis"]);
  # one_hot_encoding = one_hot_encoding.drop(columns= ["sex_female", 'chip.version_v2', "patient_patient_10", "region_cerebellum", "lab_Davis"]);
  X = sm.add_constant(one_hot_encoding);
  model = sm.OLS(expression[probe],X).fit();
  # fig = sm.qqplot(model.resid)
  # plt.show()
  white_test = het_white(model.resid,  model.model.exog)

  #define labels to use for output of White's test
  labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']

  #print results of White's test
  p = dict(zip(labels, white_test))["Test Statistic p-value"]
  if p <= alpha:
    num_sig += 1
print(num_sig)




In [ ]:
  # sorted_coef_gene = sorted(coef_gene, key=lambda tup: tup[1])
  # print(sorted_coef_gene[-10:])
  # top10genes = [tup[0] for tup in sorted_coef_gene[-10:]]
  # op_chrom = genes.loc[top10genes].loc[:, 'chrom'].values
  # op_genes = genes.loc[top10genes].loc[:, 'sym'].values

  # print(op_chrom, op_genes)

[('38446_at', 0.32211163562325, -0.6392439071327769, -0.004979364113723184), ('32052_at', 0.6618093936031971, -1.2418658692260938, -0.08175291798030049), ('38355_at', 0.8080392266143466, 0.40988997154483925, 1.206188481683854), ('41214_at', 1.198685352333169, 0.8018963739565279, 1.5954743307098103)]
['X', '11', 'Y', 'Y']
Categories (39, object): ['1', '10', '10ak*', '10ptp*', ..., 'X', 'X Y', 'Y', 'na'] ['XIST', 'HBB', 'DDX3Y', 'RPS4Y1']
Categories (8783, object): ['AADAC', 'AAK1', 'AAMP', 'AANAT', ..., 'ZYX', 'ZZEF1', 'ZZZ3', 'psiTPTE22']


## OLS per lab

In [102]:
samples

,patient,sex,region,lab,chip.version
rownames,,,,,
01_a_D_f_2.CEL,patient_01,female,A.C. cortex,Davis,v2
01_a_I_f_2.CEL,patient_01,female,A.C. cortex,Irvine,v2
01_a_M_f_1.CEL,patient_01,female,A.C. cortex,Michigan,v1
01_c_D_f_1.CEL,patient_01,female,cerebellum,Davis,v1
01_c_I_f_2.CEL,patient_01,female,cerebellum,Irvine,v2
...,...,...,...,...,...
10_c_I_f_2.CEL,patient_10,female,cerebellum,Irvine,v2
10_c_M_f_1.CEL,patient_10,female,cerebellum,Michigan,v1
10_d_D_f_2.CEL,patient_10,female,D.L.P.F. cortex,Davis,v2


In [ ]:
unique_probe_set = expression_norm.columns
p_value_gene = []
num_sig = 0
alpha = 0.05
for lab in samples.lab.unique():
  print(lab)
  probe_set_to_pvalue = {}
  for probe_set in unique_probe_set:
    lab_index = samples[samples.lab == lab].index;
    samples_tmp = samples.drop(columns= ["patient", "lab"]);
    one_hot_encoding = pd.get_dummies(samples_tmp);
    one_hot_encoding = one_hot_encoding.drop(columns= ["sex_female", 'chip.version_v2', "region_cerebellum"]);
    X = sm.add_constant(one_hot_encoding);
    model = sm.OLS(expression_norm.loc[lab_index][probe_set],X.loc[lab_index]).fit();
    p_value = model.pvalues["sex_male"];
    probe_set_to_pvalue[probe_set] = p_value

  gene_p_values = []
  for gene in genes_sym["sym"].unique():
    gene_p = 0
    gene_probes = gene_to_probe_set[gene]
    for probe in gene_probes:
      gene_p = max(gene_p, probe_set_to_pvalue[probe])
    gene_p_values.append((gene_p, gene))

  sorted_pval = sorted(gene_p_values, key=lambda tup: tup[0])
  topgenes = [tup[1] for tup in sorted_pval[:10]]
  topchrom = [genes[genes.sym == tg]["chrom"][0] for tg in topgenes]
  chrom_counter = Counter(topchrom)
  count = np.sum([chrom_counter[key] for key, value in chrom_counter.items() if key in ["X", "Y", "X Y"]])
    # print(model.pvalues["sex_male"])
  print("")




Davis
